# Convolutional encoder
#### (Python tutorial)
### Developed by Ms.Sc. Vladimir Fadeev
#### (2015-2017 GRIAT student)
### Kazan, 2019

# System initialization

In [1]:
import numpy as np
from scipy.ndimage.interpolation import shift
import itertools 

# Introduction

Main points:

- Kind of <span style="color:green">continuous</span> channel codes.

- Implemented in <span style="color:purple">2G</span>. 
Also used in <span style="color:purple">3G</span> and <span style="color:purple">4G</span> as the Turbo convolutional codes. Operated in <span style="color:purple">Deep space communications</span> as the Reed Solomon + CC concatenated codes.

- Encoders include memory registers. The number of memory registers + 1 is generally called as constrain length $L$. 
Following representation is usually used:$ (L, \mathbf{G}) $,
where $\mathbf{G}$ is the array of generators (octal form).



E.g.: $ (7, [177, 133]) $. This representation relates to following construction:


![conv](https://raw.githubusercontent.com/kirlf/communication_stuff/master/FEC/assets/conv_code_177_133.png)

Let us explain: octal **177** is the binary **1111001**, octal **133** is the binary **1011011**. So, the **one** means the existance of the link to adder (by modulo 2), **zero** - no link. 



So, let's try to implement this structure. Firstly, transform octal representation to the binary:

In [2]:
def poly2regs(poly):
    L = poly[0] # Constrain length
    Gen = [] # List for generators
    for g in poly[1]:
        bin_str = bin(int(str(g), 8))[2:] # From octal to binary representation
        split_str = list(tuple(bin_str)) # Split the bits 
        split_int = [int(i) for i in split_str] # Convert to from chars to integers
        Gen.append(split_int)
    return L, Gen

In [3]:
poly2regs([7,[171, 133]])

(7, [[1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1]])

# Python class

In [4]:
class ConvolutionalCodes:
    def __init__(self, poly):
        self.poly = poly
        
    def poly2regs(self):
        poly = self.poly
        L = poly[0] # Constrain length
        Gen = [] # List for generators
        for g in poly[1]:
            bin_str = bin(int(str(g), 8))[2:] # From octal to binary representation
            split_str = list(tuple(bin_str)) # Split the bits 
            split_int = [int(i) for i in split_str] # Convert to from chars to integers
            Gen.append(split_int)
        return L, Gen
    
    def encoder(self, message):
        L, Gen = self.poly2regs()
        Gen = np.matrix(Gen)
        memory = np.zeros((L,))
        encoded = []
        for bit in message:
            memory = np.abs(shift(memory, 1, cval=np.NaN).round())
            memory[0] = bit
            y = np.array(np.dot(Gen, memory) % 2)
            [encoded.append(i) for i in y[0]]
        return encoded          

# Run

In [5]:
convC = ConvolutionalCodes([7,[171, 133]])

In [6]:
convC.poly2regs()

(7, [[1, 1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1, 1]])

In [7]:
convC.encoder([1, 0, 1, 0, 0, 1, 1])

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0]